In [4]:
!pip install gspread pandas gspread_dataframe
import pandas as pd
import gspread
from gspread_dataframe import get_as_dataframe


In [5]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [6]:

data_path = '/content/drive/My Drive/Colab Notebooks/data'

In [7]:
import os
files = os.listdir(data_path)
print(files)

['CaseKey.csv', 'microcase2.csv']


In [8]:

import pandas as pd

# replace with your filename
df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/data/microcase2.csv")
df.head()


,Timestamp,Նշեք Ձեռ սեռը՝,Ծննդյան տարեթիվը՝ (օր.՝ 2004),Ո՞ր ոլորտից եք:,Ո՞ր մարզում եք բնակվում:,Ո՞ր ոճի երաժշտություն եք նախընտրում (կարող եք ընտրել մի քանի տարբերակ),Որքա՞ն հաճախ եք լսում դասական երաժշտություն։,Որքա՞ն հաճախ եք այցելում դասական երաժշտության համերգներին:,Որտեղի՞ց եք հիմնականում տեղեկանում դասական համերգների մասին․,Որտե՞ղ եք սովորաբար այցելում դասական երաժշտության համերգների:,Լսե՞լ եք Հայաստանի Պետական Ֆիլհարմոնիայի մասին։ (չշփոթել Հայաստանի ազգային ֆիլհարմոնիկ նվագախմբի հետ),Եղե՞լ եք Հայաստանի Պետական Ֆիլհարմոնիայի որևէ համերգին։,Ինչո՞ւ:,Ինչո՞ւ:.1,Այցելու՞մ եք համերգների:,"Որքա՞ն հետաքրքիր է Ձեզ համար դասականի և այլ ժանրերի (ջազ, ռոք, էլեկտրոնային և այլն) համադրությունը․",Ո՞ր գործոններն են ամենից շատ ազդում Ձեր՝ համերգի այցելելու ընտրության վրա։,"Որքա՞ն պատրաստ կլինեիք վճարել \nհամերգի համար , եթե այն համապատասխաներ Ձեր նախընտրություններին:"
0,9/30/2025 22:02:58,Իգական,2004.0,Տեխնիկական,Երևան,"Ջազ, Ռոք, Պոպ, Ժողովրդական (ֆոլկ), Ռեփ / Հիփ-հ...",Շաբաթը մի քանի անգամ,Երբեք,NaN,NaN,Այո,Այո,Դպրոցական/համալսարանական այց էր,NaN,Այո,հետաքրքիր,"Ծրագրի/ստեղծագործությունների բովանդակությունը,...",10000
1,9/30/2025 22:05:39,Իգական,2004.0,"Բիզնես, տնտեսագիտություն կամ մարքեթինգ",Երևան,"Դասական, Ջազ, Ռոք, Պոպ, Ռեփ / Հիփ-հոփ",Ամսվա ընթացքում մի քանի անգամ,Տարին մեկ-երկու անգամ,NaN,NaN,Ոչ,NaN,NaN,NaN,Այո,հետաքրքիր,"Ծրագրի/ստեղծագործությունների բովանդակությունը,...",9000
2,9/30/2025 22:07:51,Իգական,2004.0,Հումանիտար և սոցիալական գիտություններ,Երևան,"Դասական, Պոպ, Ռեփ / Հիփ-հոփ",Շատ հազվադեպ,NaN,NaN,NaN,Ոչ,NaN,NaN,NaN,Այո,հետաքրքիր,"Ծրագրի/ստեղծագործությունների բովանդակությունը,...",20000
3,9/30/2025 22:07:59,Իգական,2005.0,Դիզայն,Սյունիք,"Դասական, Ջազ, Ռոք, Պոպ, Էլեկտրոնային, Ռեփ / Հի...",Շատ հազվադեպ,NaN,NaN,NaN,Այո,Ոչ,NaN,Չեմ հետաքրքրվում,Այո,հետաքրքիր,"Ծրագրի/ստեղծագործությունների բովանդակությունը,...",15000
4,9/30/2025 22:10:43,Իգական,2004.0,Հումանիտար և սոցիալական գիտություններ,Երևան,Պոպ,Երբեք,NaN,NaN,NaN,Ոչ,NaN,NaN,NaN,Ոչ,Չի հետաքրքրում,Նվագախմբի կամ կատարողների որակը,5000


In [9]:
df_clean = df.rename(columns={
    'Ծննդյան տարեթիվը՝ (օր.՝ 2004) ': 'birth_year',
    'Որքա՞ն պատրաստ կլինեիք վճարել  \nհամերգի համար , եթե այն համապատասխաներ Ձեր նախընտրություններին:': 'willingness_to_pay',
    'Որքա՞ն հետաքրքիր է Ձեզ համար դասականի և այլ ժանրերի (ջազ, ռոք, էլեկտրոնային և այլն) համադրությունը․': 'genre_mix_interest'
})

In [10]:
df_clean['age'] = 2025 - pd.to_numeric(df_clean['birth_year'], errors='coerce')


In [11]:
df_clean['willingness_to_pay'] = pd.to_numeric(df_clean['willingness_to_pay'], errors='coerce')

In [12]:
# Remove outliers using IQR
q1 = df_clean['willingness_to_pay'].quantile(0.25)
q3 = df_clean['willingness_to_pay'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr

df_no_outliers = df_clean[(df_clean['willingness_to_pay'] >= lower_bound) &
                          (df_clean['willingness_to_pay'] <= upper_bound)]


In [13]:
# Mean willingness to pay without outliers
mean_wtp = df_no_outliers['willingness_to_pay'].mean()

In [14]:
interest_map = {
    'Չի հետաքրքրում': 0,
    'Չեմ լսել նախկինում': 1,
    'հետաքրքիր': 2,

}
df_clean['genre_mix_interest_num'] = df_clean['genre_mix_interest'].map(interest_map)


In [15]:
corr_age_wtp = df_no_outliers[['age', 'willingness_to_pay']].corr().iloc[0,1]
corr_age_genre = df_clean[['age', 'genre_mix_interest_num']].corr().iloc[0,1]

In [16]:
print(" Mean Willingness to Pay (excluding outliers):", round(mean_wtp, 2))
print(" Correlation between Age and Willingness to Pay:", round(corr_age_wtp, 3))
print(" Correlation between Age and Interest in Genre Mix:", round(corr_age_genre, 3))

 Mean Willingness to Pay (excluding outliers): 10824.48
 Correlation between Age and Willingness to Pay: -0.066
 Correlation between Age and Interest in Genre Mix: -0.043
